# 📱 Démonstration de l'Intégration Mobile

Ce notebook démontre les fonctionnalités d'intégration mobile de la plateforme AMU Data Science.

**Fonctionnalités couvertes :**
- Création de sessions mobiles
- Génération de QR codes
- Synchronisation audio en temps réel
- Quiz interactifs mobiles
- WebSocket et communication bidirectionnelle

## 1. Configuration et Imports

In [ ]:
import sys
from pathlib import Path
import json
import time
from datetime import datetime
import requests
from IPython.display import Image, display, HTML

# Ajouter le dossier parent au path
sys.path.append(str(Path.cwd().parent))

# Imports des modules du projet
from src.mobile_sync_manager import MobileSyncManager
from src.qr_code_generator import QRCodeGenerator
from src.real_time_interaction import RealTimeInteractionManager

print("✅ Imports réussis")

## 2. Test du Gestionnaire de Synchronisation Mobile

In [ ]:
# Initialiser le gestionnaire de synchronisation
sync_manager = MobileSyncManager(database_path='../database/amu_courses.db')

print("📱 Gestionnaire de synchronisation initialisé")

### 2.1 Créer une Session Mobile

In [ ]:
# Créer une session de test
device_info = {
    'type': 'smartphone',
    'os': 'iOS',
    'browser': 'Safari'
}

session_id = sync_manager.create_session(
    user_id='demo_user_123',
    device_info=device_info
)

print(f"✅ Session créée : {session_id}")
print(f"\n📊 État de la session :")
session_state = sync_manager.get_session_state(session_id)
print(json.dumps(session_state, indent=2))

### 2.2 Simuler la Synchronisation Audio

In [ ]:
# Simuler la lecture audio
positions = [0, 15, 30, 45, 60, 90, 120]

for position in positions:
    sync_manager.sync_audio_position(session_id, position)
    print(f"🎵 Position synchronisée : {position}s")
    time.sleep(0.5)

# Vérifier l'état mis à jour
updated_state = sync_manager.get_session_state(session_id)
print(f"\n📊 Position actuelle : {updated_state['audio_position']}s")

### 2.3 Mettre à Jour le Document en Cours

In [ ]:
# Mettre à jour le document en lecture
sync_manager.update_current_document(
    session_id=session_id,
    doc_id='M2_deep_learning_cnn',
    doc_title='CNN pour la vision par ordinateur'
)

# Vérifier la mise à jour
state = sync_manager.get_session_state(session_id)
print(f"📖 Document actuel : {state.get('current_doc_title', 'Aucun')}")

### 2.4 Activer un Quiz

In [ ]:
# Activer un quiz
quiz_data = {
    'quiz_id': 'quiz_cnn_001',
    'current_question': 1,
    'total_questions': 5,
    'score': 0
}

sync_manager.update_quiz_state(session_id, quiz_data)

# Vérifier l'état du quiz
state = sync_manager.get_session_state(session_id)
print(f"📝 Quiz actif : {state.get('quiz_active', False)}")
print(f"📊 Données du quiz : {state.get('current_quiz', {})}")

## 3. Test du Générateur de QR Codes

In [ ]:
# Initialiser le générateur de QR codes
qr_generator = QRCodeGenerator(output_dir='../mobile/static/qr_codes')

print("📱 Générateur de QR codes initialisé")

### 3.1 Générer un QR Code de Session

In [ ]:
# Générer un QR code pour la session
base_url = 'http://localhost:5000'
qr_path = qr_generator.generate_session_qr(
    session_id=session_id,
    base_url=base_url
)

print(f"✅ QR code généré : {qr_path}")
print(f"🔗 URL de la session : {base_url}/mobile/join?session={session_id}")

# Afficher le QR code
if Path(qr_path).exists():
    display(Image(filename=qr_path))
else:
    print("⚠️  Fichier QR code non trouvé")

### 3.2 Générer un QR Code pour un Cours

In [ ]:
# Générer un QR code pour un cours spécifique
course_qr_path = qr_generator.generate_course_qr(
    doc_id='M2_deep_learning_cnn',
    base_url=base_url,
    title='CNN pour la vision'
)

print(f"✅ QR code cours généré : {course_qr_path}")

# Afficher le QR code
if Path(course_qr_path).exists():
    display(Image(filename=course_qr_path))
else:
    print("⚠️  Fichier QR code non trouvé")

## 4. Test des Sessions Multiples

In [ ]:
# Créer plusieurs sessions pour simuler plusieurs utilisateurs
sessions = []

devices = [
    {'type': 'smartphone', 'os': 'iOS'},
    {'type': 'tablet', 'os': 'Android'},
    {'type': 'smartphone', 'os': 'Android'},
    {'type': 'desktop', 'os': 'Windows'}
]

for i, device in enumerate(devices, 1):
    sid = sync_manager.create_session(
        user_id=f'user_{i}',
        device_info=device
    )
    sessions.append(sid)
    print(f"✅ Session {i} créée : {sid[:8]}... ({device['type']}/{device['os']})")

print(f"\n📊 Total de sessions actives : {len(sessions)}")

### 4.1 Récupérer Toutes les Sessions Actives

In [ ]:
# Récupérer toutes les sessions actives
active_sessions = sync_manager.get_active_sessions()

print(f"📱 Sessions actives : {len(active_sessions)}")
print("\n" + "="*60)

for session in active_sessions:
    print(f"\nSession ID : {session['session_id'][:12]}...")
    print(f"  User : {session['user_id']}")
    print(f"  Device : {session['device_type']} ({session['device_os']})")
    print(f"  Started : {session['started_at']}")
    print(f"  Active : {session['is_active']}")

### 4.2 Filtrer les Sessions par Utilisateur

In [ ]:
# Récupérer les sessions d'un utilisateur spécifique
user_sessions = sync_manager.get_active_sessions(user_id='user_1')

print(f"📱 Sessions de user_1 : {len(user_sessions)}")
for session in user_sessions:
    print(f"  - {session['session_id'][:12]}... ({session['device_type']})")

## 5. Test de l'API Mobile (si l'application est lancée)

### 5.1 Créer une Session via l'API

In [ ]:
# URL de l'API (assurez-vous que l'application Flask est lancée)
API_URL = 'http://localhost:5000'

try:
    # Créer une session via l'API
    response = requests.post(
        f'{API_URL}/mobile/create-session',
        json={
            'user_id': 'api_test_user',
            'device_info': {
                'type': 'smartphone',
                'os': 'iOS'
            }
        },
        timeout=5
    )
    
    if response.status_code == 200:
        data = response.json()
        print("✅ Session créée via API")
        print(f"\nSession ID : {data['session_id']}")
        print(f"QR Code URL : {data['qr_code_url']}")
        print(f"Join URL : {data['join_url']}")
        
        # Afficher le QR code si disponible
        qr_url = f"{API_URL}{data['qr_code_url']}"
        display(HTML(f'<img src="{qr_url}" width="300">'))
    else:
        print(f"❌ Erreur : {response.status_code}")
        print(response.text)
        
except requests.exceptions.ConnectionError:
    print("⚠️  L'application Flask n'est pas lancée")
    print("💡 Lancez l'application avec : python app.py")
except Exception as e:
    print(f"❌ Erreur : {e}")

### 5.2 Vérifier le Statut de l'API Mobile

In [ ]:
try:
    # Vérifier le statut de santé de l'application
    response = requests.get(f'{API_URL}/health', timeout=5)
    
    if response.status_code == 200:
        health_data = response.json()
        print("✅ Application en ligne")
        print(f"\nStatut : {health_data['status']}")
        print(f"Timestamp : {health_data['timestamp']}")
        print("\nServices :")
        for service, status in health_data['services'].items():
            icon = "✅" if status else "❌"
            print(f"  {icon} {service}")
    else:
        print(f"❌ Erreur : {response.status_code}")
        
except requests.exceptions.ConnectionError:
    print("⚠️  L'application Flask n'est pas lancée")
    print("💡 Lancez l'application avec : python app.py")
except Exception as e:
    print(f"❌ Erreur : {e}")

## 6. Simulation de Synchronisation en Temps Réel

In [ ]:
# Simuler plusieurs appareils synchronisés
import random

print("🎵 Simulation de synchronisation audio entre appareils...\n")

# Créer 3 sessions
sync_sessions = []
for i in range(3):
    sid = sync_manager.create_session(
        user_id=f'sync_user_{i}',
        device_info={'type': 'smartphone', 'os': 'iOS'}
    )
    sync_sessions.append(sid)

# Simuler la synchronisation
for position in range(0, 61, 10):
    print(f"\n⏱️  Position : {position}s")
    
    for i, sid in enumerate(sync_sessions, 1):
        # Ajouter un petit décalage aléatoire pour simuler la latence réseau
        actual_position = position + random.uniform(-0.5, 0.5)
        sync_manager.sync_audio_position(sid, int(actual_position))
        
        state = sync_manager.get_session_state(sid)
        print(f"  📱 Appareil {i} : {state['audio_position']}s")
    
    time.sleep(1)

print("\n✅ Simulation terminée")

## 7. Nettoyage et Fermeture des Sessions

In [ ]:
# Fermer toutes les sessions de test
all_test_sessions = sessions + sync_sessions + [session_id]

print("🧹 Nettoyage des sessions de test...\n")

for sid in all_test_sessions:
    sync_manager.close_session(sid)
    print(f"❌ Session fermée : {sid[:12]}...")

# Vérifier qu'il ne reste plus de sessions actives
remaining = sync_manager.get_active_sessions()
print(f"\n📊 Sessions actives restantes : {len(remaining)}")

if len(remaining) == 0:
    print("✅ Toutes les sessions de test ont été fermées")
else:
    print("⚠️  Il reste des sessions actives")

## 8. Résumé et Conclusion

In [ ]:
print("="*60)
print("📱 RÉSUMÉ DE LA DÉMONSTRATION MOBILE")
print("="*60)
print("\n✅ Fonctionnalités testées :")
print("  1. ✅ Création de sessions mobiles")
print("  2. ✅ Synchronisation de position audio")
print("  3. ✅ Mise à jour de document en cours")
print("  4. ✅ Activation de quiz")
print("  5. ✅ Génération de QR codes")
print("  6. ✅ Gestion de sessions multiples")
print("  7. ✅ Simulation de synchronisation temps réel")
print("  8. ✅ Nettoyage de sessions")
print("\n🎯 Prochaines étapes :")
print("  - Lancez l'application : python app.py")
print("  - Testez l'interface mobile via QR code")
print("  - Essayez la synchronisation entre plusieurs appareils")
print("="*60)

## 9. Bonus : Génération de QR Codes en Masse

In [ ]:
# Générer des QR codes pour tous les cours disponibles
from src.course_indexer import CourseIndexer

try:
    # Initialiser l'indexeur
    indexer = CourseIndexer(
        course_materials_path='../data/course_materials',
        index_db_path='../database/amu_courses.db'
    )
    
    # Récupérer tous les cours
    courses = indexer.get_all_documents()
    
    print(f"📚 Génération de QR codes pour {len(courses)} cours...\n")
    
    # Générer un QR code pour chaque cours
    for course in courses[:5]:  # Limiter à 5 pour la démo
        qr_path = qr_generator.generate_course_qr(
            doc_id=course['doc_id'],
            base_url='http://localhost:5000',
            title=course['title']
        )
        print(f"✅ {course['title'][:50]}...")
    
    print(f"\n✅ QR codes générés dans : mobile/static/qr_codes/")
    
except Exception as e:
    print(f"⚠️  Erreur : {e}")
    print("💡 Assurez-vous d'avoir exécuté : python scripts/index_existing_courses.py")

## 10. Instructions pour Tester sur Mobile

### Étapes pour tester l'intégration mobile :

1. **Lancez l'application Flask :**
   ```bash
   python app.py
   ```

2. **Créez une session mobile :**
   - Exécutez la cellule 5.1 ci-dessus
   - Un QR code sera généré

3. **Scannez le QR code avec votre téléphone :**
   - Utilisez l'appareil photo de votre smartphone
   - Ou une application de scan de QR code

4. **Testez les fonctionnalités :**
   - Lecture audio synchronisée
   - Quiz interactifs
   - Synchronisation entre plusieurs appareils

5. **Vérifiez la synchronisation :**
   - Ouvrez la même session sur plusieurs appareils
   - Testez la synchronisation de la position audio
   - Essayez les quiz en temps réel